# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from pickle import PicklingError
from joblib import dump, load
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.metrics import make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, recall_score, classification_report, f1_score, accuracy_score, precision_score, confusion_matrix, matthews_corrcoef, roc_curve, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector as selector
from datetime import datetime
from tensorflow import keras
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.utils.data import evaluate_print
from IPython.display import display, Markdown
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Settings
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "device_organization"
TIME_BUCKETS  = [
    "7d", 
    "2d", 
    "1d", 
    "12h",
    "6h", 
    "1h"
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def add_count_and_threat(x):
    threat = 1 if 1 in x.threat.values else 0
    return pd.Series([len(x), threat], index=["count", "threat"])


def train_classifier_on_single_users(df, classifier, time_bucket, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a clone of the classifier
        try:
            classifier = clone(classifier)
        except:
            pass

        # Update the parameters of the classifier according to the grid search
        classifier.set_params(**params)
        
        # Set the random state attribute of the classifier (if it has one)
        try:
            classifier.set_params(**{"random_state": randint(0, 2**32)})
        except Exception:
            pass

        # Define the transformers that do the rest of the preprocessing (scaling, encoding)
        numeric_transformer = Pipeline(steps=[
            ("scaler", RobustScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ("ohe", OneHotEncoder())
        ])

        # Create a pipeline that performs the feature selection and scaling
        pipe = Pipeline([
            ("column_transformer", ColumnTransformer(
                transformers=[
                    ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                    ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                ]
            )),
            ("classifier", classifier)
        ])
       
        # Benchmark the training
        start_training = timer()
        pipe.fit(df[["count"]])
        end_training = timer()

        # Benchmark the inference
        start_inference = timer()
        pipe.predict(df[["count"]])
        end_inference = timer()

        # Calculate different evaluation metrics
        y_true = df["threat"]
        y_pred = pipe.named_steps["classifier"].labels_
        scores = pipe.named_steps["classifier"].decision_scores_
        recall = recall_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(y_true, scores)
            p_auc_10 = roc_auc_score(y_true, scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(y_true, scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(y_true, scores, max_fpr=0.3)
        except ValueError:
            auc = -1

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": end_training - start_training,
                "inference_time": end_inference - start_inference,
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": y_true,
                "y_pred": y_pred,
                "scores": scores
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, time_bucket, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.time_bucket = time_bucket
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        time_bucket=self.time_bucket,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, time_bucket in enumerate(TIME_BUCKETS, start=1):

        display(Markdown(f"# {i}/{len(TIME_BUCKETS)} - Bucket: {time_bucket}"))

        # Group the dataframe by users and time buckets
        tmp = df.groupby([df["user"], pd.Grouper(freq=time_bucket, key="date")]).apply(add_count_and_threat)
        tmp = tmp.reindex(pd.MultiIndex.from_product(tmp.index.levels))
        tmp = tmp.fillna(0)

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=tmp,
            classifier=classifier,
            time_bucket=time_bucket,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "device"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 24 different hyperparameter combinations
 Training Time: 0.0269    Inference Time: 0.0096    pAUC: 0.5340 ± 0.0263   Recall: 0.0011 ± 0.0022   TN: 18867    FP: 34      FN: 178   TP: 1     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 0.2028    Inference Time: 0.0465    pAUC: 0.5616 ± 0.0169   Recall: 0.0067 ± 0.0042   TN: 18731    FP: 170     FN: 177   TP: 2     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 0.6351    Inference Time: 0.2133    pAUC: 0.5638 ± 0.0064   Recall: 0.0112 ± 0.0000   TN: 18771    FP: 130     FN: 177   TP: 2     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 1.1587    Inference Time: 0.4289    pAUC: 0.5597 ± 0.0054   Recall: 0.0112 ± 0.0000   TN: 18731    FP: 170     FN: 177   TP: 2     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.0284    Inference Time: 0.0102    pAUC: 0.5293 ± 0.0452   Recall: 0.0056 ± 0.0061   TN: 18723    FP: 178     FN: 176   TP: 3     Params: {'max_sample

# 2/6 - Bucket: 2d

Testing 24 different hyperparameter combinations
 Training Time: 0.0581    Inference Time: 0.0253    pAUC: 0.5507 ± 0.0591   Recall: 0.0015 ± 0.0031   TN: 65990    FP: 0       FN: 260   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 0.4031    Inference Time: 0.1460    pAUC: 0.5580 ± 0.0315   Recall: 0.0131 ± 0.0108   TN: 65408    FP: 582     FN: 251   TP: 9     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 1.5586    Inference Time: 0.6736    pAUC: 0.5567 ± 0.0101   Recall: 0.0185 ± 0.0132   TN: 65437    FP: 553     FN: 258   TP: 2     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 3.0498    Inference Time: 1.3504    pAUC: 0.5597 ± 0.0096   Recall: 0.0277 ± 0.0178   TN: 65429    FP: 561     FN: 251   TP: 9     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.0559    Inference Time: 0.0242    pAUC: 0.6046 ± 0.0682   Recall: 0.0362 ± 0.0381   TN: 65533    FP: 457     FN: 238   TP: 22    Params: {'max_sample

# 3/6 - Bucket: 1d

...

Testing 24 different hyperparameter combinations
 Training Time: 0.1687    Inference Time: 0.0756    pAUC: 0.5783 ± 0.0195   Recall: 0.0017 ± 0.0033   TN: 263728   FP: 794     FN: 478   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 1.0792    Inference Time: 0.4813    pAUC: 0.5742 ± 0.0088   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 4.9261    Inference Time: 2.2923    pAUC: 0.5702 ± 0.0100   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 9.6897    Inference Time: 4.6140    pAUC: 0.5747 ± 0.0051   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.1762    Inference Time: 0.0795    pAUC: 0.5697 ± 0.0093   Recall: 0.0033 ± 0.0041   TN: 264522   FP: 0       FN: 478   TP: 0     Params: {'max_sample

# 5/6 - Bucket: 6h

Testing 24 different hyperparameter combinations
 Training Time: 0.3084    Inference Time: 0.1385    pAUC: 0.6750 ± 0.0185   Recall: 0.0041 ± 0.0058   TN: 526955   FP: 2239    FN: 533   TP: 8     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 1.9064    Inference Time: 0.8911    pAUC: 0.6834 ± 0.0097   Recall: 0.0218 ± 0.0099   TN: 526505   FP: 2689    FN: 533   TP: 8     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 8.9758    Inference Time: 4.2687    pAUC: 0.6800 ± 0.0077   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 17.7935   Inference Time: 8.4246    pAUC: 0.6839 ± 0.0038   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.3090    Inference Time: 0.1395    pAUC: 0.6704 ± 0.0100   Recall: 0.0041 ± 0.0058   TN: 528622   FP: 572     FN: 541   TP: 0     Params: {'max_sample

# 6/6 - Bucket: 1h

Testing 24 different hyperparameter combinations
 Training Time: 1.5504    Inference Time: 0.6901    pAUC: 0.8370 ± 0.0052   Recall: 0.0439 ± 0.0424   TN: 3176791   FP: 0       FN: 1354   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 9.2308    Inference Time: 4.3803    pAUC: 0.8356 ± 0.0080   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 43.6951   Inference Time: 21.0867   pAUC: 0.8356 ± 0.0081   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 87.3398   Inference Time: 42.0339   pAUC: 0.8389 ± 0.0066   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 1.5964    Inference Time: 0.7209    pAUC: 0.8394 ± 0.0068   Recall: 0.0861 ± 0.0287   TN: 3169400   FP: 7391    FN: 1311   TP: 43    Params: {'

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 28 different hyperparameter combinations
 Training Time: 0.0573    Inference Time: 0.0329    pAUC: 0.4810 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18771    FP: 130     FN: 177   TP: 2     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.1004    Inference Time: 0.0574    pAUC: 0.4810 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18771    FP: 130     FN: 177   TP: 2     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 0.1443    Inference Time: 0.0899    pAUC: 0.4810 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18771    FP: 130     FN: 177   TP: 2     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 0.1964    Inference Time: 0.1105    pAUC: 0.4810 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18771    FP: 130     FN: 177   TP: 2     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.0542    Inference Time: 0.0311    pAUC: 0.4816 ± 0.0000   Recall: 0.0056 ± 0.0000   TN: 18867    FP: 34      FN: 178   TP: 1     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 2/6 - Bucket: 2d

Testing 28 different hyperparameter combinations
 Training Time: 0.1876    Inference Time: 0.1127    pAUC: 0.4842 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65795    FP: 195     FN: 259   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 0.3494    Inference Time: 0.2016    pAUC: 0.4959 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65795    FP: 195     FN: 259   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 0.5194    Inference Time: 0.3016    pAUC: 0.4842 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65795    FP: 195     FN: 259   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 0.6906    Inference Time: 0.4158    pAUC: 0.4842 ± 0.0000   Recall: 0.0038 ± 0.0000   TN: 65795    FP: 195     FN: 259   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.1808    Inference Time: 0.1135    pAUC: 0.4889 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65581    FP: 409     FN: 258   TP: 2     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 3/6 - Bucket: 1d

...

Testing 28 different hyperparameter combinations
 Training Time: 0.6810    Inference Time: 0.3985    pAUC: 0.5879 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 263728   FP: 794     FN: 478   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 1.3281    Inference Time: 0.8017    pAUC: 0.5879 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 263728   FP: 794     FN: 478   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 1.9947    Inference Time: 1.2145    pAUC: 0.5879 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 263728   FP: 794     FN: 478   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 2.6644    Inference Time: 1.6085    pAUC: 0.5879 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 263728   FP: 794     FN: 478   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 0.6665    Inference Time: 0.4205    pAUC: 0.5843 ± 0.0000   Recall: 0.0021 ± 0.0000   TN: 263627   FP: 895     FN: 477   TP: 1     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Bucket: 6h

Testing 28 different hyperparameter combinations
 Training Time: 1.2564    Inference Time: 0.7826    pAUC: 0.6887 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 2.5352    Inference Time: 1.5457    pAUC: 0.6887 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 3.7598    Inference Time: 2.3124    pAUC: 0.6887 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 4.9790    Inference Time: 3.1073    pAUC: 0.6887 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 1.2683    Inference Time: 0.7938    pAUC: 0.6854 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Bucket: 1h

Testing 28 different hyperparameter combinations
 Training Time: 4.8908    Inference Time: 3.4140    pAUC: 0.8316 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 9.8939    Inference Time: 6.7707    pAUC: 0.8316 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 14.5473   Inference Time: 10.3647   pAUC: 0.8316 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 19.1845   Inference Time: 13.7825   pAUC: 0.8316 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 4.9724    Inference Time: 3.5419    pAUC: 0.8343 ± 0.0000   Recall: 0.0318 ± 0.0000   TN: 3169400   FP: 7391    FN: 1311   TP: 43    Params: {'n_bins': 8, 'n_random_c

# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 0.0173    Inference Time: 0.0178    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.4794 ± 0.0000
Recall: 0.0112 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 0.0337    Inference Time: 0.0481    pAUC: 0.4954 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.4954 ± 0.0000
Recall: 0.0077 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 0.0952    Inference Time: 0.1680    pAUC: 0.5859 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5859 ± 0.0000
Recall: 0.0084 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 0.1729    Inference Time: 0.3022    pAUC: 0.6891 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6891 ± 0.0000
Recall: 0.0296 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 0.9760    Inference Time: 1.9631    pAUC: 0.8422 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8422 ± 0.0000
Recall: 0.0938 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 1 different hyperparameter combinations
 Training Time: 0.0173    Inference Time: 0.0179    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.4794 ± 0.0000
Recall: 0.0112 ± 0.0000


# 2/6 - Bucket: 2d

Testing 1 different hyperparameter combinations
 Training Time: 0.0317    Inference Time: 0.0481    pAUC: 0.4954 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.4954 ± 0.0000
Recall: 0.0077 ± 0.0000


# 3/6 - Bucket: 1d

...

Testing 1 different hyperparameter combinations
 Training Time: 0.0960    Inference Time: 0.1755    pAUC: 0.5859 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5859 ± 0.0000
Recall: 0.0084 ± 0.0000


# 5/6 - Bucket: 6h

Testing 1 different hyperparameter combinations
 Training Time: 0.1723    Inference Time: 0.3034    pAUC: 0.6891 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6891 ± 0.0000
Recall: 0.0296 ± 0.0000


# 6/6 - Bucket: 1h

Testing 1 different hyperparameter combinations
 Training Time: 0.9770    Inference Time: 1.8545    pAUC: 0.8422 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.8422 ± 0.0000
Recall: 0.0938 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [1, 2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 1.2550    Inference Time: 0.0182    pAUC: 0.6677 ± 0.0000   Recall: 0.1117 ± 0.0000   TN: 18782.0   FP: 119.0   FN: 159.0   TP: 20.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 0.4364    Inference Time: 0.0356    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721.0   FP: 180.0   FN: 177.0   TP: 2.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 0.9429    Inference Time: 0.0351    pAUC: 0.5130 ± 0.0000   Recall: 0.0503 ± 0.0000   TN: 18738.0   FP: 163.0   FN: 170.0   TP: 9.0   Params: {'alpha':

# 2/6 - Bucket: 2d

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 0.7191    Inference Time: 0.0347    pAUC: 0.6448 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 65990.0   FP: 0.0     FN: 260.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 0.5082    Inference Time: 0.0468    pAUC: 0.4954 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478.0   FP: 512.0   FN: 258.0   TP: 2.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 1.0161    Inference Time: 0.0480    pAUC: 0.4818 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478.0   FP: 512.0   FN: 258.0   TP: 2.0   Params: {'alpha': 

# 3/6 - Bucket: 1d

...

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 1.3627    Inference Time: 0.0723    pAUC: 0.5437 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 264522.0   FP: 0.0     FN: 478.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 1.3564    Inference Time: 0.0909    pAUC: 0.5859 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664.0   FP: 1858.0   FN: 474.0   TP: 4.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 2.1848    Inference Time: 0.0728    pAUC: 0.5623 ± 0.0119   Recall: 0.0469 ± 0.0234   TN: 262989.0   FP: 1533.0   FN: 450.0   TP: 28.0   Params: {'al

# 5/6 - Bucket: 6h

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 0.7151    Inference Time: 0.0513    pAUC: 0.5958 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 529194.0   FP: 0.0     FN: 541.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 0.6859    Inference Time: 0.0458    pAUC: 0.6891 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218.0   FP: 4976.0   FN: 525.0   TP: 16.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 1.3409    Inference Time: 0.0549    pAUC: 0.6199 ± 0.0071   Recall: 0.0000 ± 0.0000   TN: 529194.0   FP: 0.0     FN: 541.0   TP: 0.0   Params: {'alp

# 6/6 - Bucket: 1h

Testing 120 different hyperparameter combinations
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': True}: Could not form valid cluster separation. Please change n_clusters or change clustering method
Skipping {'alpha': 0.2, 'beta': 2, 'n_clusters': 1, 'use_weights': False}: Could not form valid cluster separation. Please change n_clusters or change clustering method
 Training Time: 3.3555    Inference Time: 0.2680    pAUC: 0.8205 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 3176791.0   FP: 0.0     FN: 1354.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 3.2731    Inference Time: 0.2441    pAUC: 0.8422 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518.0   FP: 21273.0   FN: 1227.0   TP: 127.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 6.8670    Inference Time: 0.2753    pAUC: 0.8397 ± 0.0004   Recall: 0.0009 ± 0.0018   TN: 3176282.0   FP: 509.0   FN: 1348.0   TP: 6.0   Param

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 168 different hyperparameter combinations
 Training Time: 0.0166    Inference Time: 0.0050    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.0132    Inference Time: 0.0049    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.0121    Inference Time: 0.0044    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.0121    Inference Time: 0.0044    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {'n_components': 1, 'stan

# 2/6 - Bucket: 2d

Testing 168 different hyperparameter combinations
 Training Time: 0.0164    Inference Time: 0.0068    pAUC: 0.4795 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.0178    Inference Time: 0.0061    pAUC: 0.4795 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.0161    Inference Time: 0.0060    pAUC: 0.4795 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.0156    Inference Time: 0.0057    pAUC: 0.4795 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {'n_components': 1, 'stan

# 3/6 - Bucket: 1d

...

Testing 168 different hyperparameter combinations
 Training Time: 0.0384    Inference Time: 0.0165    pAUC: 0.5250 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.0382    Inference Time: 0.0168    pAUC: 0.5250 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.0377    Inference Time: 0.0170    pAUC: 0.5250 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.0383    Inference Time: 0.0166    pAUC: 0.5250 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'n_components': 1, 'stan

# 5/6 - Bucket: 6h

Testing 168 different hyperparameter combinations
 Training Time: 0.0678    Inference Time: 0.0282    pAUC: 0.6418 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.0685    Inference Time: 0.0289    pAUC: 0.6418 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.0781    Inference Time: 0.0362    pAUC: 0.6418 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.0688    Inference Time: 0.0297    pAUC: 0.6418 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'n_components': 1, 'stan

# 6/6 - Bucket: 1h

Testing 168 different hyperparameter combinations
 Training Time: 0.3899    Inference Time: 0.1215    pAUC: 0.7539 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.3770    Inference Time: 0.1178    pAUC: 0.7539 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.3703    Inference Time: 0.1147    pAUC: 0.7539 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.3616    Inference Time: 0.1089    pAUC: 0.7539 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'n_components': 

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 7.5536    Inference Time: 0.6275    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 8.4388    Inference Time: 0.6771    pAUC: 0.4794 ± 0.0000   Recall: 0.0112 ± 0.0000   TN: 18721    FP: 180     FN: 177   TP: 2     Params: {'hidden_activation': 'sigmoid', '

# 2/6 - Bucket: 2d

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 13.0934   Inference Time: 1.9805    pAUC: 0.4954 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 17.7662   Inference Time: 2.3952    pAUC: 0.4954 ± 0.0000   Recall: 0.0077 ± 0.0000   TN: 65478    FP: 512     FN: 258   TP: 2     Params: {'hidden_activation': 'sigmoid', '

# 3/6 - Bucket: 1d

...

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 43.7168   Inference Time: 9.2481    pAUC: 0.5859 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 53.9508   Inference Time: 8.7281    pAUC: 0.5860 ± 0.0000   Recall: 0.0084 ± 0.0000   TN: 262664   FP: 1858    FN: 474   TP: 4     Params: {'hidden_activation': 'sigmoid', '

# 5/6 - Bucket: 6h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 82.6135   Inference Time: 18.3982   pAUC: 0.6891 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 93.1870   Inference Time: 18.1697   pAUC: 0.6891 ± 0.0000   Recall: 0.0296 ± 0.0000   TN: 524218   FP: 4976    FN: 525   TP: 16    Params: {'hidden_activation': 'sigmoid', '

# 6/6 - Bucket: 1h

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 361.4759   Inference Time: 110.7354   pAUC: 0.8422 ± 0.0000   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}: The number of neurons should not exceed the number of features
 Training Time: 560.4171   Inference Time: 99.7276   pAUC: 0.8245 ± 0.0353   Recall: 0.0938 ± 0.0000   TN: 3155518   FP: 21273   FN: 1227   TP: 127   Params: {'hidden_activation': 'sigm

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Bucket: 7d

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 5.4509    Inference Time: 0.5871    pAUC: 0.5784 ± 0.0733   Recall: 0.0346 ± 0.0498   TN: 18734.0   FP: 167.0   FN: 155.0   TP: 24.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_2" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_2/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_2/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • y=arr

# 2/6 - Bucket: 2d

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 11.9576   Inference Time: 2.1167    pAUC: 0.5317 ± 0.0819   Recall: 0.0215 ± 0.0297   TN: 65358.0   FP: 632.0   FN: 239.0   TP: 21.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_38" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_38/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_38/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  • y=

# 3/6 - Bucket: 1d

...

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 45.9805   Inference Time: 8.2866    pAUC: 0.5986 ± 0.0254   Recall: 0.0172 ± 0.0176   TN: 263045.0   FP: 1477.0   FN: 453.0   TP: 25.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_110" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_110/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_110/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
 

# 5/6 - Bucket: 6h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 76.0890   Inference Time: 15.3898   pAUC: 0.6955 ± 0.0133   Recall: 0.0237 ± 0.0118   TN: 529193.0   FP: 1.0     FN: 541.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_146" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_146/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_146/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  •

# 6/6 - Bucket: 1h

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 428.2334   Inference Time: 119.2088   pAUC: 0.6679 ± 0.1079   Recall: 0.0764 ± 0.0371   TN: 3155518.0   FP: 21273.0   FN: 1227.0   TP: 127.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_182" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_182/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_182/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=floa